In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import geopandas as gpd

def truncate_by_threshold(paths, flux_column='flux', threshold=.99):
    paths_sorted = paths.reset_index(drop=True).sort_values(by=flux_column, ascending=False)
    fluxes_sorted = paths_sorted[flux_column]
    total_flux = fluxes_sorted.sum()
    flux_percentiles = fluxes_sorted.cumsum() / total_flux
    excess = flux_percentiles[flux_percentiles >= threshold]
    cutoff = excess.idxmin()
    keep = flux_percentiles[flux_percentiles <= threshold].index
    paths_truncated = paths_sorted.loc[keep, :]
    return paths_truncated, fluxes_sorted, flux_percentiles, keep, cutoff


def plot_path_truncation(fluxes_sorted, flux_percentiles, cutoff, threshold):
    
    fig, axs = plt.subplots(1, 2, figsize=(10, 3))

    ax = axs[0]
    ax.plot(fluxes_sorted.values, flux_percentiles.values, alpha=.8, linewidth=.5, color='black')
    ax.axvline(x=fluxes_sorted[cutoff], linestyle='--', color='red', linewidth=.5)
    ax.axhline(y=threshold, linestyle='--', color='red', linewidth=.5, alpha=.8)
    ax.fill_between(x=fluxes_sorted, y1=flux_percentiles, where=fluxes_sorted>fluxes_sorted[cutoff], alpha=.8, color="w", edgecolor='black', hatch='//')
    ax.invert_xaxis()
    ax.set_xscale('log')
    ax.set_xlabel('Log(flux)')
    ax.set_ylabel('Cumulative density')

    ax = axs[1]
    fluxes_sorted.hist(ax=ax, alpha=.8, edgecolor='black', color='white', hatch='//')
    ax.axvline(x=fluxes_sorted[cutoff], linestyle='--', color='red', linewidth=.5)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.invert_xaxis()
    ax.set_ylabel('Number of observations (log)')
    ax.set_xlabel('Flux (log)')
    ax.grid(False);

    plt.suptitle('Choice of cutoff based-on fluxes')
    
    return fig

In [2]:
loss_cols = ['trips_lost_amin', 'trips_lost_mean', 'trips_lost_amax',
       '%trips_lost_amin', '%trips_lost_mean', '%trips_lost_amax',
       'time_m_delta_amin', 'time_m_delta_mean', 'time_m_delta_amax',
       '%time_m_delta_amin', '%time_m_delta_mean', '%time_m_delta_amax',
       'rerouting_loss_person_time_m_amin',
       'rerouting_loss_person_time_m_mean',
       'rerouting_loss_person_time_m_amax']

In [4]:
paths = pd.read_parquet("/Users/alison/Documents/World Bank Caribbean/data/results/transport/path and flux data/LCA_pathdata_time_m_30.parquet", engine='fastparquet')

In [23]:
threshold = .99
paths_truncated, fluxes_sorted, flux_percentiles, keep, cutoff = truncate_by_threshold(paths, threshold=threshold)
fig = plot_path_truncation(fluxes_sorted, flux_percentiles, cutoff, threshold)
fig.savefig("/Users/alison/Documents/World Bank Caribbean/data/figures/thresholding.png", dpi=300, bbox_inches='tight')

# Looking at data

In [7]:
fail = pd.read_parquet("/Users/alison/Documents/World Bank Caribbean/data/results/transport/disruption results/lca_roads_edges_sector_damages_with_roads.parquet")

In [8]:
loss_cols = ['trips_lost_amin', 'trips_lost_mean', 'trips_lost_amax',
       '%trips_lost_amin', '%trips_lost_mean', '%trips_lost_amax',
       'time_m_delta_amin', 'time_m_delta_mean', 'time_m_delta_amax',
       '%time_m_delta_amin', '%time_m_delta_mean', '%time_m_delta_amax',
       'rerouting_loss_person_time_m_amin',
       'rerouting_loss_person_time_m_mean',
       'rerouting_loss_person_time_m_amax']

fail[loss_cols].describe()

,trips_lost_amin,trips_lost_mean,trips_lost_amax,%trips_lost_amin,%trips_lost_mean,%trips_lost_amax,time_m_delta_amin,time_m_delta_mean,time_m_delta_amax,%time_m_delta_amin,%time_m_delta_mean,%time_m_delta_amax,rerouting_loss_person_time_m_amin,rerouting_loss_person_time_m_mean,rerouting_loss_person_time_m_amax
count,97.000000,249.000000,249.000000,97.000000,249.000000,249.000000,97.000000,249.000000,249.000000,97.000000,249.000000,249.000000,97.000000,249.000000,249.000000
mean,25564.826313,10272.223915,10272.223915,0.262304,0.106755,0.106755,42358.360325,20861.939959,20861.939959,0.527731,0.580616,0.580616,7509.667547,3135.195106,3135.195106
std,14158.005930,15081.269891,15081.269891,0.135773,0.150463,0.150463,15468.728488,19715.941117,19715.941117,0.193525,0.137760,0.137760,2460.717875,3822.134813,3822.134813
min,3522.194446,430.997347,430.997347,0.039410,0.006859,0.006859,13904.326468,6897.062755,6897.062755,0.127862,0.127862,0.127862,3390.350417,264.821069,264.821069
25%,17593.725942,478.240304,478.240304,0.197217,0.007110,0.007110,35669.780825,7116.041848,7116.041848,0.362090,0.503315,0.503315,6018.657860,285.898164,285.898164
50%,24869.139567,576.016254,576.016254,0.267835,0.008050,0.008050,47378.955005,7274.739430,7274.739430,0.503315,0.585503,0.585503,7859.935306,380.972179,380.972179
75%,39132.168692,17596.075033,17596.075033,0.389664,0.197222,0.197222,50974.836476,47378.955005,47378.955005,0.756451,0.659669,0.659669,9468.866696,7859.935306,7859.935306
max,52355.389312,52355.389312,52355.389312,0.453517,0.453517,0.453517,60306.054218,60306.054218,60306.054218,0.812477,0.812477,0.812477,10337.451111,10337.451111,10337.451111


In [9]:
fail[['rp', 'hazard'] + loss_cols]

,rp,hazard,trips_lost_amin,trips_lost_mean,trips_lost_amax,%trips_lost_amin,%trips_lost_mean,%trips_lost_amax,time_m_delta_amin,time_m_delta_mean,time_m_delta_amax,%time_m_delta_amin,%time_m_delta_mean,%time_m_delta_amax,rerouting_loss_person_time_m_amin,rerouting_loss_person_time_m_mean,rerouting_loss_person_time_m_amax
0,NaN,landslide,52355.389312,52355.389312,52355.389312,0.453517,0.453517,0.453517,13904.326468,13904.326468,13904.326468,0.127862,0.127862,0.127862,4204.016161,4204.016161,4204.016161
1,100.0,coastal,3850.751814,3850.751814,3850.751814,0.042111,0.042111,0.042111,15567.406168,15567.406168,15567.406168,0.758022,0.758022,0.758022,3390.350417,3390.350417,3390.350417
2,10.0,coastal,3522.194446,3522.194446,3522.194446,0.039410,0.039410,0.039410,15870.577432,15870.577432,15870.577432,0.812477,0.812477,0.812477,3455.443264,3455.443264,3455.443264
3,50.0,coastal,3793.897442,3793.897442,3793.897442,0.041254,0.041254,0.041254,15679.924983,15679.924983,15679.924983,0.773224,0.773224,0.773224,3403.770066,3403.770066,3403.770066
4,5.0,coastal,3522.194446,3522.194446,3522.194446,0.039410,0.039410,0.039410,15870.577432,15870.577432,15870.577432,0.812477,0.812477,0.812477,3455.443264,3455.443264,3455.443264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,5.0,pluvial,17593.725942,17596.075033,17596.075033,0.197217,0.197222,0.197222,60306.054218,60306.054218,60306.054218,0.756451,0.756435,0.756435,10337.451111,10337.451111,10337.451111
245,100.0,pluvial,39132.168692,39132.168692,39132.168692,0.389664,0.389664,0.389664,47378.955005,47378.955005,47378.955005,0.362090,0.362090,0.362090,7859.935306,7859.935306,7859.935306
246,10.0,pluvial,24869.139567,24869.139567,24869.139567,0.267835,0.267835,0.267835,50974.836476,50974.836476,50974.836476,0.503315,0.503315,0.503315,9468.866696,9468.866696,9468.866696
247,50.0,pluvial,39132.168692,39132.168692,39132.168692,0.389664,0.389664,0.389664,47378.955005,47378.955005,47378.955005,0.362090,0.362090,0.362090,7859.935306,7859.935306,7859.935306


In [13]:
fail.groupby(['rp', 'hazard']).mean()[loss_cols]

/var/folders/7w/k72v6f5x25gblsqmh6bxd3j40000gn/T/ipykernel_7138/1985423000.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fail.groupby(['rp', 'hazard']).mean()[loss_cols]


trips_lost_amin  trips_lost_mean  trips_lost_amax  \
rp    hazard                                                                  
5.0   coastal                 3662.585344      3662.585344      3662.585344   
      fluvial_defended                NaN       430.997347       430.997347   
      fluvial_undefended              NaN       478.240304       478.240304   
      pluvial                18742.475462     18744.453644     18744.453644   
10.0  coastal                 3784.097064      3784.097064      3784.097064   
      fluvial_defended                NaN       443.429704       443.429704   
      fluvial_undefended              NaN       478.240304       478.240304   
      pluvial                24869.139567     24869.139567     24869.139567   
50.0  coastal                 3947.266256      3947.266256      3947.266256   
      fluvial_defended                NaN       544.490067       544.490067   
      fluvial_undefended              NaN       529.350982       529.350982   
      pluvial                39132.168692     39132.168692     39132.168692   
100.0 coastal                 4022.669042      4022.669042      4022.669042   
      fluvial_defended                NaN       599.840547       599.840547   
      fluvial_undefended              NaN       598.464266       598.464266   
      pluvial                40958.830728     40958.830728     40958.830728   

                          %trips_lost_amin  %trips_lost_mean  \
rp    hazard                                                   
5.0   coastal                     0.040440          0.040440   
      fluvial_defended                 NaN          0.006859   
      fluvial_undefended               NaN          0.007110   
      pluvial                     0.208367          0.208371   
10.0  coastal                     0.041528          0.041528   
      fluvial_defended                 NaN          0.006925   
      fluvial_undefended               NaN          0.007110   
      pluvial                     0.267835          0.267835   
50.0  coastal                     0.043442          0.043442   
      fluvial_defended                 NaN          0.007695   
      fluvial_undefended               NaN          0.007559   
      pluvial                     0.389664          0.389664   
100.0 coastal                     0.044501          0.044501   
      fluvial_defended                 NaN          0.008333   
      fluvial_undefended               NaN          0.008321   
      pluvial                     0.404684          0.404684   

                          %trips_lost_amax  time_m_delta_amin  \
rp    hazard                                                    
5.0   coastal                     0.040440       15762.184563   
      fluvial_defended            0.006859                NaN   
      fluvial_undefended          0.007110                NaN   
      pluvial                     0.208371       58832.704048   
10.0  coastal                     0.041528       15678.203279   
      fluvial_defended            0.006925                NaN   
      fluvial_undefended          0.007110                NaN   
      pluvial                     0.267835       50974.836476   
50.0  coastal                     0.043442       15604.224857   
      fluvial_defended            0.007695                NaN   
      fluvial_undefended          0.007559                NaN   
      pluvial                     0.389664       47378.955005   
100.0 coastal                     0.044501       15654.513813   
      fluvial_defended            0.008333                NaN   
      fluvial_undefended          0.008321                NaN   
      pluvial                     0.404684       41832.504077   

                          time_m_delta_mean  time_m_delta_amax  \
rp    hazard                                                     
5.0   coastal                  15762.184563       15762.184563   
      fluvial_defended          6993.120434        6993.120434   
      fluvial_u

In [15]:
import seaborn

fg = seaborn.factorplot(x='hazard', y='trips_lost_mean', hue='rp', 
                        col='hazard', data=fail, kind='bar')
fg.set_xlabels('')

AttributeError: module 'seaborn' has no attribute 'factorplot'

In [ ]:
for rp in [5, 10, 50, 100]:
    fail_rp = fail[fail['rp'] == rp]
    for 

In [ ]:
rp = 5
fail_rp = fail[fail['rp'] == rp]


In [30]:
data = pd.read_parquet("/Users/alison/Documents/World Bank Caribbean/data/processed_data/infrastructure/transport/LCA_schools_pathdata_time_m_60.parquet", engine='fastparquet')

In [31]:
data

,origin_id,destination_id,edge_path,time_m,flux,school_district
0,roadn_10054,LCA_edu_12,"[roade_475, roade_474, roade_473, roade_472, r...",32.022526,0.001965,Five
1,roadn_10054,LCA_edu_126,"[roade_475, roade_474, roade_473, roade_472, r...",14.334686,0.008314,Five
2,roadn_10054,LCA_edu_127,"[roade_475, roade_474, roade_473, roade_472, r...",29.663314,0.003880,Five
3,roadn_10054,LCA_edu_128,"[roade_475, roade_474, roade_473, roade_472, r...",29.718432,0.003604,Five
4,roadn_10054,LCA_edu_149,"[roade_475, roade_474, roade_473, roade_472, r...",30.934998,0.000078,Five
...,...,...,...,...,...,...
5273,roadn_9933,LCA_edu_39_and_LCA_edu_63,[roade_11030],0.311917,7.128984,Three
5274,roadn_9933,LCA_edu_50,"[roade_11030, roade_26344, roade_34638, roade_...",5.345988,0.001133,Three
5275,roadn_9933,LCA_edu_6,"[roade_11029, roade_11028, roade_11027, roade_...",1.581064,0.028880,Three
5276,roadn_9933,LCA_edu_61_and_LCA_edu_115,"[roade_11029, roade_11028, roade_11027, roade_...",4.622602,0.002202,Three
